## loading necessary libraries

In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Operator
import numpy as np

# Matrix of the multi-controlled NOT gate

In [2]:
def UMCX(n: int):
    return np.array([[1 if (i == j and i < (1 << (n + 1)) - 2) or [i, j] == [(1 << (n + 1)) - 1, (1 << (n + 1)) - 2] or [i, j] == [(1 << (n + 1)) - 2, (1 << (n + 1)) - 1] else 0 for i in range(1 << (n + 1))] for j in range(1 << (n + 1))])

# Construction of the multi-controlled NOT gate

In [5]:
def MCX(n: int):

    QRegs = QuantumRegister(n + 1, "q")
    QC = QuantumCircuit(QRegs)

    QControl, QTarget = QRegs[1:], QRegs[0]

    # QC.h(QTarget) in terms of U gates
    QC.u(np.pi / 2, 0, np.pi, QTarget)

    # Step 1.
    for i, q in enumerate(reversed(QControl[1:])):
        # QC.cp(alpha * np.exp2(- (i + 1)), q, QTarget) in terms of U and CNOT gates
        QC.u(0, 0, np.pi * np.exp2(- (i + 2)), q)
        QC.u(0, 0, np.pi * np.exp2(- (i + 2)), QTarget)
        QC.cx(q, QTarget)
        QC.u(0, 0, - np.pi * np.exp2(- (i + 2)), QTarget)
        QC.cx(q, QTarget)
    
        for j, p in enumerate(QControl[len(QControl) - i:]):
            # QC.crx(np.pi * np.exp2(- (j + 1)), q, p) in terms of U and CNOT gates
            QC.u(0, 0, np.pi / 2, p)
            QC.cx(q, p)
            QC.u(- np.pi * np.exp2(- (j + 2)), 0, 0, p)
            QC.cx(q, p)
            QC.u(np.pi * np.exp2(- (j + 2)), - np.pi / 2, 0, p)

    # Step 2.
    # QC.cp(alpha * np.exp2(1 - len(QControl)), QControl[0], QTarget) in terms of U and CNOT gates
    QC.u(0, 0, np.pi * np.exp2(- len(QControl)), QControl[0])
    QC.u(0, 0, np.pi * np.exp2(- len(QControl)), QTarget)
    QC.cx(QControl[0], QTarget)
    QC.u(0, 0, - np.pi * np.exp2(- len(QControl)), QTarget)
    QC.cx(QControl[0], QTarget)

    for i, q in enumerate(QControl[1:]):
        # QC.crx(np.pi * np.exp2(- i), QControl[0], q) in terms of U and CNOT gates
        QC.u(0, 0, np.pi / 2, q)
        QC.cx(QControl[0], q)
        QC.u(- np.pi * np.exp2(- (i + 1)), 0, 0, q)
        QC.cx(QControl[0], q)
        QC.u(np.pi * np.exp2(- (i + 1)), - np.pi / 2, 0, q)

    # Step 3.
    for i, q in enumerate(QControl[1:]):
        # QC.cp(- alpha * np.exp2(i + 1 - len(QControl)), q, QTarget) in terms of U and CNOT gates
        QC.u(0, 0, - np.pi * np.exp2(i - len(QControl)), q)
        QC.u(0, 0, - np.pi * np.exp2(i - len(QControl)), QTarget)
        QC.cx(q, QTarget)
        QC.u(0, 0, np.pi * np.exp2(i - len(QControl)), QTarget)
        QC.cx(q, QTarget)

        for j, p in enumerate(QControl[i + 2:]):
            # QC.crx(- np.pi * np.exp2(- (j + 1)), q, p) in terms of U and CNOT gates
            QC.u(0, 0, np.pi / 2, p)
            QC.cx(q, p)
            QC.u(np.pi * np.exp2(- (j + 2)), 0, 0, p)
            QC.cx(q, p)
            QC.u(- np.pi * np.exp2(- (j + 2)), - np.pi / 2, 0, p)

    # Step 4.
    for i, q in enumerate(reversed(QControl[1:])):
        for j, p in enumerate(QControl[len(QControl) - i:]):
            # QC.crx(np.pi * np.exp2(- (j + 1)), q, p) in terms of U and CNOT gates
            QC.u(0, 0, np.pi / 2, p)
            QC.cx(q, p)
            QC.u(- np.pi * np.exp2(- (j + 2)), 0, 0, p)
            QC.cx(q, p)
            QC.u(np.pi * np.exp2(- (j + 2)), - np.pi / 2, 0, p)

    # Step 5.
    for i, q in enumerate(QControl[1:]):
        # QC.crx(- np.pi * np.exp2(- i), QControl[0], q) in terms of U and CNOT gates
        QC.u(0, 0, np.pi / 2, q)
        QC.cx(QControl[0], q)
        QC.u(np.pi * np.exp2(- (i + 1)), 0, 0, q)
        QC.cx(QControl[0], q)
        QC.u(- np.pi * np.exp2(- (i + 1)), - np.pi / 2, 0, q)
    
    # Step 6.
    for i, q in enumerate(QControl[1:]):
        for j, p in enumerate(QControl[i + 2:]):
            # QC.crx(- np.pi * np.exp2(- (j + 1)), q, p) in terms of U and CNOT gates
            QC.u(0, 0, np.pi / 2, p)
            QC.cx(q, p)
            QC.u(np.pi * np.exp2(- (j + 2)), 0, 0, p)
            QC.cx(q, p)
            QC.u(- np.pi * np.exp2(- (j + 2)), - np.pi / 2, 0, p)

    # QC.h(QTarget) in terms of U gates
    QC.u(np.pi / 2, 0, np.pi, QTarget)

    return QC

In [6]:
np.linalg.norm(np.round(np.array(Operator(MCX(2))) - UMCX(2), 1))

0.0

# Testing

In [7]:
N = 8
for n in range(2, N + 1):
    QC = MCX(n)
    U_gate, depth = np.array(Operator(QC)), QC.depth
    U_matrix = UMCX(n)
    print(f"the quadratic (l2) rounding error for n = {n} is {100 * np.linalg.norm(U_gate - U_matrix)/(n + 1)}%")
    print(f"circuit depth = {QC.depth()}")

the quadratic (l2) rounding error for n = 2 is 2.9146197658401005e-14%
circuit depth = 22
the quadratic (l2) rounding error for n = 3 is 4.213080496118039e-14%
circuit depth = 46
the quadratic (l2) rounding error for n = 4 is 1.131538518235862e-13%
circuit depth = 76
the quadratic (l2) rounding error for n = 5 is 1.2077651746554987e-13%
circuit depth = 112
the quadratic (l2) rounding error for n = 6 is 1.3653447098732616e-13%
circuit depth = 154
the quadratic (l2) rounding error for n = 7 is 1.918710586982283e-13%
circuit depth = 202
the quadratic (l2) rounding error for n = 8 is 2.703583809460644e-13%
circuit depth = 256


# Remark:
The following code is a generalization of the above one and it implements a multi-controlled phase gate that scales polynomially with the number of qubits, as opposed to qiskit's own decomposition whose gate count, circuit depth, and CNOT count are all const. * 2**n

In [12]:
def MCX(n: int):

    QRegs = QuantumRegister(n + 1, "q")
    QC = QuantumCircuit(QRegs)

    QControl, QTarget = QRegs[1:], QRegs[0]
    for i, q in enumerate(reversed(QControl[1:])):
        for j, p in enumerate(QControl[len(QControl) - i:]):
            QC.crx(np.pi * np.exp2(- (j + 1)), q, p)
        QC.cp(alpha * np.exp2(- (i + 1)), q, QTarget)

    for i, q in enumerate(QControl[1:]):
        QC.crx(np.pi * np.exp2(- i), QControl[0], q)
        QC.cp(- alpha * np.exp2(i + 1 - len(QControl)), q, QTarget)
        for j, p in enumerate(QControl[i + 2:]):
            QC.crx(- np.pi * np.exp2(- (j + 1)), q, p)

    QC.cp(alpha * np.exp2(1 - len(QControl)), QControl[0], QTarget)

    for i, q in enumerate(reversed(QControl[1:])):
        for j, p in enumerate(QControl[len(QControl) - i:]):
            QC.crx(np.pi * np.exp2(- (j + 1)), q, p)

    for i, q in enumerate(QControl[1:]):
        QC.crx(- np.pi * np.exp2(- i), QControl[0], q)
        for j, p in enumerate(QControl[i + 2:]):
            QC.crx(- np.pi * np.exp2(- (j + 1)), q, p)
    
    return QC